In [29]:
import time
import os
import numpy as np
import random

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram

from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI

QI_URL = os.getenv('API_URL', 'https://api.quantum-inspire.com/')

In [30]:
authentication = get_authentication()
QI.set_authentication(authentication, QI_URL)

Enter email:
jmascaror98@gmail.com
Enter password
········


In [31]:
def BB84Circuit(nbits, AliceBasis, EveBasis, BobBasis, message, bitPosition):
   
    """
    Circuit Generator
    
    nbits: Self-explainatory
    AliceBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    EveBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    BobBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    message: nqubits-length binary array
    bitPosition: index of the bit in message to be communicated
    """
    q = QuantumRegister(1)
    ans = ClassicalRegister(1)
    qc = QuantumCircuit(q, ans)
    if message[bitPosition] == 1:
        qc.x(q[0])
    qc.barrier()
    
# Man-In-The-Middle-Attack

    had_ind = (AliceBasis[bitPosition] + EveBasis[bitPosition])%2

    if had_ind == 1:    
        qc.h(q[0])
    qc.barrier()
    
# Eve measures

    qc.measure(q[0],ans[0])
    
# Bob measures

    if (had_ind+BobBasis[bitPosition])%2 == 1:
        qc.h(q[0])

    qc.measure(q[0],ans[0])
    
    return q, qc

In [44]:
nbits = 10
messageInABottle = []
AliceBasis = []
BobBasis = []
EveBasis = []
for i in range(nbits):
    
    messageInABottle.append(random.randint(0,1))
    AliceBasis.append(random.randint(0,1))
    BobBasis.append(random.randint(0,1))
    EveBasis.append(random.randint(0,1))

print("Message: ", messageInABottle)
print("AliceBasis ", AliceBasis)
print("EveBasis ", EveBasis )
print("BobBasis ", BobBasis)


Message:  [0, 1, 1, 0, 0, 1, 0, 1, 0, 0]
AliceBasis  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
EveBasis  [0, 0, 0, 1, 1, 1, 0, 1, 1, 0]
BobBasis  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0]


In [48]:
def BB84Message(nbits,AliceBasis,EveBasis,BobBasis, message):
    """
    Constructs the Message with Quantum Magic
    nbits: Self-explainatory
    AliceBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    EveBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    BobBasis: nqubits-length array with entries 0 (z basis) or 1 (x basis)
    message: nqubits-length binary array
    
    returns 
    constrMess (constructed message)
    sameBasisIndices (bit positions in which Alice and Bob use the same basis)
    
    """
    AliceEveMess = []
    constrMess = []
    sameBasisIndicesA_E = []
    sameBasisIndices = []
    for bitPos in range(nbits):
        print("Quantum Communicating bit number: ",bitPos+1,"/",nbits)
        q, qc = BB84Circuit(nbits,AliceBasis,EveBasis,BobBasis,message,bitPos)
        qi_backend = QI.get_backend('QX-34-L')
        job = execute(qc, qi_backend)
        result = job.result()
        counts = result.get_counts()
        
        
        if (AliceBasis[bitPos] + EveBasis[bitPos])%2 == 1: #different basis: info destructed
            AliceEveMess.append("E_x")
            
        else: #same basis
            sameBasisIndicesA_E.append(bitPos)
            if len(counts)==1: #only one result
                AliceEveMess.append(list(counts.keys())[0]) #add that result to the message
                
            else: #get the result with more counts
                if counts['0'] >= counts['1']:
                    AliceEveMess.append('0')
                else:
                    AliceEveMess.append('1')
                    
        if (AliceBasis[bitPos] + BobBasis[bitPos])%2 == 1: #different basis: info destructed
            constrMess.append("x")
            
        else: #same basis
            sameBasisIndices.append(bitPos)
            if len(counts)==1: #only one result
                constrMess.append(list(counts.keys())[0]) #add that result to the message
                
            else: #get the result with more counts
                if counts['0'] >= counts['1']:
                    constrMess.append('0')
                else:
                    constrMess.append('1')
        
    
    return AliceEveMess, sameBasisIndicesA_E, constrMess, sameBasisIndices    
        

In [49]:
AliceEveMes, goodIndices_A_E, outputMes, goodIndices = BB84Message(nbits,AliceBasis,EveBasis,BobBasis,messageInABottle)

Quantum Communicating bit number:  1 / 10
Quantum Communicating bit number:  2 / 10
Quantum Communicating bit number:  3 / 10


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  4 / 10


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  5 / 10


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  6 / 10


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  7 / 10
Quantum Communicating bit number:  8 / 10
Quantum Communicating bit number:  9 / 10
Quantum Communicating bit number:  10 / 10


In [59]:
print("Alice-Eve Message: ",AliceEveMes)
print("Output Message: ", outputMes)
print("Message: ", messageInABottle)

Alice-Eve Message:  ['E_x', '1', 'E_x', 'E_x', '0', 'E_x', '0', 'E_x', 'E_x', '0']
Output Message:  ['0', '1', 'x', '0', '0', '0', '0', 'x', 'x', '0']
Message:  [0, 1, 1, 0, 0, 1, 0, 1, 0, 0]


In [51]:
print(goodIndices_A_E)
print(goodIndices)

[1, 4, 6, 9]
[0, 1, 3, 4, 5, 6, 9]


In [53]:
def BB84KeyConstruction(message,output,goodIndices,prop,tolerance):
    """
    message: Alice starting message
    output: Bob's result through Quantum Magic
    goodIndices: bit Positions in which both basis are the same    
    prop: Proportion of common good bits to be compared
    tolerance: minimum success rate to generate a safe QK

    """
    auxList = []    
    nGoodBits = len(goodIndices)
    
    
    nComparingBits = int(nGoodBits*prop)
    comparingBitPositions = random.sample(goodIndices, nComparingBits)
    print("Bit Positions to compare: ",comparingBitPositions)
    
    allOkCounter = 0
    for i in range(nComparingBits):
        j = comparingBitPositions[i]
        if int(message[j])== int(output[j]):
            allOkCounter +=1
            
    for i in range(nGoodBits):
        if i not in comparingBitPositions:
            auxList.append(output[goodIndices[i]])
    
    print(allOkCounter)
    successRate = float(allOkCounter/nComparingBits)
    print("Success rate: ",successRate)
    if successRate < tolerance:
        print("Failure")
        return 0
    else:
        print("Success!!!")
        return auxList

    

In [55]:
prop = 0.5
tolerance = 0.8

QKD = BB84KeyConstruction(messageInABottle,outputMes,goodIndices,prop,tolerance)
print(QKD)

Bit Positions to compare:  [9, 6, 3]
3
Success rate:  1.0
Success!!!
['0', '1', '0', '0', '0']


In [62]:
def wholeBB84Protocol(nbits,prop,tolerance):
    messageInABottle = []
    AliceBasis = []
    BobBasis = []
    EveBasis = []
    for i in range(nbits):

        messageInABottle.append(random.randint(0,1))
        AliceBasis.append(random.randint(0,1))
        BobBasis.append(random.randint(0,1))
        EveBasis.append(random.randint(0,1))

    print("Message: ", messageInABottle)
    print("AliceBasis ", AliceBasis)
    print("EveBasis ", EveBasis)
    print("BobBasis ", BobBasis)
    
    
    AliceEveMes, goodIndices_A_E, outputMes, goodIndices = BB84Message(nbits,AliceBasis,EveBasis,BobBasis,messageInABottle)
    print("Message: ", messageInABottle)
    print("Output Message: ",outputMes)
    print("Same Basis Instances: ",goodIndices)
    
    QKD = BB84KeyConstruction(messageInABottle,outputMes,goodIndices,prop,tolerance)
    print("Quantum Key Generated", QKD)
    
    return QKD
    
    

In [63]:
nbits = 12
prop = 0.3
tolerance = 1
Key = wholeBB84Protocol(nbits,prop,tolerance)

Message:  [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]
AliceBasis  [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]
EveBasis  [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0]
BobBasis  [1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
Quantum Communicating bit number:  1 / 12


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  2 / 12


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  3 / 12


...

Quantum Communicating bit number:  7 / 12
Quantum Communicating bit number:  8 / 12


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Quantum Communicating bit number:  9 / 12
Quantum Communicating bit number:  10 / 12
Quantum Communicating bit number:  11 / 12
Quantum Communicating bit number:  12 / 12


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


Message:  [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]
Output Message:  ['x', '1', 'x', 'x', 'x', '0', '1', '0', 'x', '0', 'x', 'x']
Same Basis Instances:  [1, 5, 6, 7, 9]
Bit Positions to compare:  [1]
0
Success rate:  0.0
Failure
Quantum Key Generated 0
